In [ ]:
# pypylon 
import pypylon.pylon as py
# plotting for graphs and display of image
import pypylon.genicam as geni

import matplotlib.pyplot as plt
# linear algebra and basic math on image matrices
import numpy as np
# OpenCV for image processing functions
import time
from imageio import get_writer
import os
import sys
import threading, queue
from collections import deque
import multiprocessing as mp
from campy.cameras.basler import cam
from campy import CampyParams
from campy.writer import campipe
from campy.display import display
import argparse
import ast
import yaml
import copy

In [ ]:
# Recording parameters
params = {}
params["videoFolder"] = "./test"
# camSettings: ["./videoSettings/Cylinder_front_acA640-750um_22636055.pfs", "./videoSettings/Cylinder_side_acA720-520um_40190567.pfs","./videoSettings/Cylinder_top_acA800-510um_23193015"

params["camSettings"] =  ["./videoSettings/Cylinder_front_acA640-750um_22636055.pfs","./videoSettings/Cylinder_top_acA800-510um_23193015.pfs", "./videoSettings/Cylinder_side_acA720-520um_40190567.pfs"]
params["frameRate"]   = 50
params["recTimeInSec"] = 40

# Camera parameters
params["cameraMake"] = "basler"
params["numCams"] = 3
params["cameraNames"]= ['Camera1','Camera2','Camera3']

# Compression parameters
params["ffmpeg_path"]=[]
params["gpus"] = [0,0,0]
params["pixelFormatInput"] = "gray"
params["pixelFormatOutput"] =  "gray"
params["quality"] = "21"

# Display parameters
params["chunkLengthInSec"] = 30
params["displayFrameRate"] = 10
params["displayDownsample"] =  2

In [ ]:
print(params)

In [ ]:
def create_cam_params(params, n_cam):
    # Insert camera-specific metadata from parameters into cam_params dictionary
    cam_params = copy.deepcopy(params)
    cam_params["camSettings"] = params["camSettings"][n_cam]
    cam_params["n_cam"] = n_cam
    cam_params["cameraName"] = params["cameraNames"][n_cam]
    cam_params["gpu"] = params["gpus"][n_cam]
    cam_params["baseFolder"] = os.getcwd()

    # Other metadata
    # date
    # time
    # hardware?
    # os?

    return cam_params

In [ ]:
def acquire_one_camera(n_cam):
    # Initializes metadata dictionary for this camera stream
    # and inserts important configuration details

    # cam_params["cameraMake"] == "basler":
    #     from campy.cameras.basler import cam

    cam_params = create_cam_params(params, n_cam)

    # Open camera n_cam
    camera, cam_params = cam.Open(cam_params)

    # Initialize queues for display window and video writer
    writeQueue = deque()
    stopQueue = deque([], 1)

    # Start image window display ('consumer' thread)
    if sys.platform == 'win32' and cam_params["cameraMake"] == "basler":
        dispQueue = []
    else:
        dispQueue = deque([], 2)
        threading.Thread(
            target=display.DisplayFrames,
            daemon=True,
            args=(cam_params, dispQueue,),
        ).start()

    # Start grabbing frames ('producer' thread)
    threading.Thread(
        target = cam.GrabFrames,
        daemon=True,
        args = (cam_params,
                camera,
                writeQueue,
                dispQueue,
                stopQueue),
        ).start()

    # Start video file writer (main 'consumer' thread)
    campipe.WriteFrames(cam_params, writeQueue, stopQueue)

In [ ]:
cam_params = create_cam_params(params, 1)

In [ ]:
print(cam_params)

In [ ]:
# Open camera n_cam
camera, cam_params = cam.Open(cam_params)

In [ ]:
# Initialize queues for display window and video writer
writeQueue = deque()
stopQueue = deque([], 1)
# Start image window display ('consumer' thread)
if sys.platform == 'win32' and cam_params["cameraMake"] == "basler":
     dispQueue = []
else:
    dispQueue = deque([], 2)
    threading.Thread(
        target=display.DisplayFrames,
         daemon=True,
        args=(cam_params, dispQueue,),
    ).start()

# Start grabbing frames ('producer' thread)
threading.Thread(
    target = cam.GrabFrames,
    daemon=True,
    args = (cam_params,
            camera,
            writeQueue,
            dispQueue,
            stopQueue),
    ).start()

# Start video file writer (main 'consumer' thread)
campipe.WriteFrames(cam_params, writeQueue, stopQueue)

In [ ]:
cam_array.StopGrabbing()

In [ ]:
# store a unique number for each camera to identify the incoming images
for idx, cam in enumerate(cam_array):
    ModelName = cam.DeviceInfo.GetModelName()
    print(f"set context {idx} for camera {ModelName}")
    cam.SetCameraContext(idx)
    window_name = f'Camera-{idx:03}'
    params["camSettings"] = ["./videoSettings/Cylinder_front_acA640-750um_22636055.pfs", "./videoSettings/Cylinder_side_acA720-520um_40190567.pfs","./videoSettings/Cylinder_top_acA800-510um_23193015"

                             params["n_cam"] = []
    params["cameraName"] = ['Camera1', 'Camera2', 'Camera3']
    params["gpu"] = []
    params[]    
    cam_params = params
    cam_params["n_cam"] = n_cam
    cam_params["cameraName"] = params["cameraNames"][n_cam]
    cam_params["gpu"] = params["gpus"][n_cam]
    cam_params["baseFolder"] = os.getcwd()


In [ ]:
# set the exposure time for each camera
for idx, cam in enumerate(cam_array):
    ModelName = cam.DeviceInfo.GetModelName()
    print(f"set Exposuretime {idx} for camera {ModelName}")
    cam.ExposureTime.SetValue(6000)
    print(cam.ExposureTime.Unit)
    print(cam.ExposureTime.Value)
    print(cam.ExposureTime.Min)
    print(cam.ExposureTime.Max)
    cam.ChunkModeActive = True
    cam.ChunkSelector = "LineStatusAll"
    cam.ChunkEnable = True
    # get clean powerup state
    cam.UserSetSelector = "Default"
    cam.UserSetLoad.Execute()

    #
    cam.LineSelector = "Line4"
    cam.LineMode = "Input"

    cam.TriggerSelector = "FrameStart"
    cam.TriggerSource = "Line4"
    cam.TriggerMode = "On"
    cam.TriggerActivation.Value
    print(cam.Height())
    print(cam.Width())

In [ ]:
writer = get_writer(
       'output-filename.mkv',  # mkv players often support H.264
        fps=50,  # FPS is in units Hz; should be real-time.
        codec='libx264',  # When used properly, this is basically "PNG for video" (i.e. lossless)
        quality=None,  # disables variable compression
        ffmpeg_params=[  # compatibility with older library versions
            '-preset',   # set to fast, faster, veryfast, superfast, ultrafast
            'fast',      # for higher speed but worse compression
            '-crf',      # quality; set to 0 for lossless, but keep in mind
            '24'         # that the camera probably adds static anyway
        ]
)

In [ ]:
vid_cod = cv2.VideoWriter_fourcc('X','V','I','D')
output = cv2.VideoWriter('C:\Users\WangLab\Documents\VideoData\test.avi', vid_cod, 50.0, (480, 640))
# Loop until you hit the Esc key
while True:
    # Capture the current frame
    ret, frame = cap.read()
    # Display the image
    if ret:
        cv2.imshow('Webcam', frame)
        output.write(frame)
    # Detect if the Esc key has been pressed
    c = cv2.waitKey(1)
    if c == 27:
        break
# Release the video capture object
cap.release()
output.release()
# Close all active windows
cv2.destroyAllWindows()

In [ ]:
vid_cod = cv2.VideoWriter_fourcc('X','V','I','D')
output = cv2.VideoWriter('C:\\Users\\WangLab\\Documents\\VideoData\\test.mp4', vid_cod, 50.0, (480, 640))

In [ ]:
# setup named window per camera


frame_counts = [0]*NUM_CAMERAS
frames_to_grab = 20
cam_array.StartGrabbing(1)
c = 0
while True:
    with cam_array.RetrieveResult(10000) as res:
        if res.GrabSucceeded():
            img_nr = res.ImageNumber
            cam_id = res.GetCameraContext()
            frame_counts[cam_id] = img_nr
            #print(f"cam #{cam_id}  image #{img_nr}")
            #height = res.Height
            #print(height)
            # do something with the image ....
            img = res.Array
            window_name = f'Camera-{cam_id:03}'
            cv2.imshow(window_name, img)
            if cam_id ==0:
                output.write(img)
            print(res.BlockID, end='\r')
            res.Release()
            # check if all cameras have reached 100 images
            k = cv2.waitKey(1)
            if k == 27:
                break

                
# Close all active windows
output.release()
cv2.destroyAllWindows()               
cam_array.StopGrabbing()


In [ ]:

output = cv2.VideoWriter('C:\\anaconda\\Codes\\SA13_033122.avi', vid_cod, 30.0, (640,480))
# Loop until you hit the Esc key
while True:
    # Capture the current frame
    ret, frame = cap.read()
    # Display the image
    if ret:
        cv2.imshow('Webcam', frame)
        output.write(frame)
    # Detect if the Esc key has been pressed
    c = cv2.waitKey(1)
    if c == 27:
        break
# Release the video capture object
cap.release()
output.release()
# Close all active windows
cv2.destroyAllWindows()


In [ ]:
output.release()

In [ ]:
cam_array.StopGrabbing()

In [ ]:
# open the camera
tlf = py.TlFactory.GetInstance()
cam = py.InstantCamera(tlf.CreateFirstDevice())
cam.Open()

In [ ]:
# enable the chunk that
# samples all IO lines on every FrameStart
cam.ChunkModeActive = True
cam.ChunkSelector = "LineStatusAll"
cam.ChunkEnable = True

In [ ]:
cam.ChunkSelector.Symbolics

In [ ]:
# set max speed
cam.Height = cam.Height.Min
cam.Width = cam.Width.Min
cam.ExposureTime = cam.ExposureTime.Min

# limit to 1khz
cam.AcquisitionFrameRateEnable = True
cam.AcquisitionFrameRate = 1000

print( cam.ResultingFrameRate.Value)
cam.StartGrabbingMax(1000)

io_res = []
while cam.IsGrabbing():
    with cam.RetrieveResult(1000) as res:
        time_stamp = res.TimeStamp
        io_res.append((time_stamp, res.ChunkLineStatusAll.Value))
        
cam.StopGrabbing()

In [ ]:
# list of timestamp + io status
io_res[:10]

In [ ]:
# simple logic analyzer :-)

# convert to numpy array
io_array = np.array(io_res)
# extract first column timestamps
x_vals = io_array[:,0]
#  start with first timestamp as '0'
x_vals -= x_vals[0]

# extract second column io values
y_vals = io_array[:,1]
# for each bit plot the graph
for bit in range(8):
    
    logic_level = ((y_vals & (1<<bit)) != 0)*0.8 +bit
    # plot in seconds
    plt.plot(x_vals / 1e9, logic_level, label = bit)
    
plt.xlabel("time [s]")
plt.ylabel("IO_LINE [#]")
plt.legend()
plt.show()

In [ ]:
# get clean powerup state
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

#
cam.LineSelector = "Line4"
cam.LineMode = "Input"

cam.TriggerSelector = "FrameStart"
cam.TriggerSource = "Line4"
cam.TriggerMode = "On"
cam.TriggerActivation.Value

In [ ]:
# definition of event handler class 
class TriggeredImage(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.grab_times = []
    def OnImageGrabbed(self, camera, grabResult):
        self.grab_times.append(grabResult.TimeStamp)

In [ ]:
# create event handler instance
image_timestamps = TriggeredImage()

# register handler
# remove all other handlers
cam.RegisterImageEventHandler(image_timestamps, 
                              py.RegistrationMode_ReplaceAll, 
                              py.Cleanup_None)

# start grabbing with background loop
cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
# wait ... or do something relevant
while cam.IsGrabbing():
    time.sleep(0.1)
# stop grabbing
cam.StopGrabbing()

In [ ]:
frame_delta_s = np.diff(np.array(image_timestamps.grab_times))/1.e9
plt.plot(frame_delta_s, ".")
plt.axhline(np.mean(frame_delta_s))

In [ ]:
plt.hist(frame_delta_s  - np.mean(frame_delta_s) , bins=100)
plt.xticks(rotation=45)
plt.show()

In [ ]:
cam.Close()